In [2]:
import pandas as pd
import numpy as np
import itertools
import os
import re
import requests
import json

# Gather

In [41]:
# pull standard cards from mtgjson
standard_sets = requests.get('https://www.mtgjson.com/files/StandardPrintings.json')
standard_sets_clean = standard_sets.json()

# code to save standard_sets as a json file 
with open('standard_sets_clean.txt', 'w') as outfile:
    json.dump(standard_sets_clean, outfile)

# code to load standard_sets_clean
with open('standard_sets_clean.txt', encoding='utf-8') as file:
    standard_sets_clean = json.load(file)

In [2]:
decks = pd.read_csv('C:/Users/muroc/Documents/MTG/data/decks.csv')

FileNotFoundError: [Errno 2] File b'C:/Users/muroc/Documents/MTG/data/decks.csv' does not exist: b'C:/Users/muroc/Documents/MTG/data/decks.csv'

In [6]:
decks = pd.read_csv('/home/steadyjingo/Documents/MTGA_rarity_analysis/data/mtg_data.csv')

In [15]:
decks_clean = decks.copy()

In [2]:
decks_clean = pd.read_csv('~/Documents/MTGA_rarity_analysis/data/decks_clean.csv')

# Assess

In [10]:
decks.drop(columns=['Unnamed: 0'], inplace=True)
decks.head()

,deck_list,player,url,name
0,"['2 Forest', '1 Island', '1 Mountain', '2 Swam...",Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control
1,"['4 Shock', '1 Forest', '1 Island', '1 Mountai...",Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals
2,"['3 Forest', '1 Island', '2 Plains', '1 Swamp'...",Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value
3,"['1 Island', '1 Mountain', '2 Plains', '3 Swam...",Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_
4,"['1 Island', '1 Mountain', '2 Plains', '3 Swam...",Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23442&d=361922...,Mardu_DOOM


In [16]:
decks_clean.tail()

,deck_list,player,url,name
488,"['4 Planar Cleansing', '7 Island', '5 Plains',...",Yokokawa_Yuuta,https://www.mtgtop8.com/event?e=23907&d=366416...,UW_Control
489,"['1 Disenchant', '7 Island', '4 Plains', '4 Ha...",zer1997,https://www.mtgtop8.com/event?e=23772&d=365157...,UW_Control
490,"['17 Plains', '4 Loyal Pegasus', '4 Faerie Gui...",Brandon_Krussow,https://www.mtgtop8.com/event?e=23943&d=366839...,White_Weenie
491,"['2 Disenchant', '17 Plains', '4 Loyal Pegasus...",Nematov_Ruslan,https://www.mtgtop8.com/event?e=23578&d=363291...,White_Weenie
492,NaN,Aki_Vainio,https://www.mtgtop8.com/event?e=23996&d=367291...,Rakdos_Knights


In [17]:
decks_clean.sample(10)

,deck_list,player,url,name
115,"['2 Disenchant', '2 Island', '2 Mountain', '1 ...",Brett_Girvan,https://www.mtgtop8.com/event?e=23953&d=366920...,Fires_of_Invention
264,"['8 Forest', '6 Swamp', '2 Epic Downfall', '4 ...",David_Pound,https://www.mtgtop8.com/event?e=23943&d=366838...,Lucky_Golgari
240,"['6 Forest', '3 Swamp', '4 Blood Crypt', '4 Ca...",bnjy99,https://www.mtgtop8.com/event?e=23793&d=365328...,Jund_Aggro
122,"['1 Forest', '1 Island', '1 Mountain', '1 Plai...",Nathan_Basser,https://www.mtgtop8.com/event?e=23953&d=366923...,Fires_of_Invention
471,"['1 Forest', '2 Island', '2 Mountain', '4 Bree...",Oono_Isao,https://www.mtgtop8.com/event?e=23907&d=366412...,Temur_Reclamation
336,"['6 Mountain', '9 Swamp', '4 Blood Crypt', '4 ...",Dmitriy_Butakov,https://www.mtgtop8.com/event?e=23942&d=366787...,Rakdos_Sacrifice
428,"['5 Forest', '1 Island', '3 Swamp', '4 Breedin...",Chantelle_Campbell,https://www.mtgtop8.com/event?e=23753&d=364983...,Sultai_Oven
313,"['4 Shock', '18 Mountain', '4 Bonecrusher Gian...",Quinn_Kotecki,https://www.mtgtop8.com/event?e=23922&d=366542...,Mono_Red
333,"['6 Mountain', '9 Swamp', '4 Blood Crypt', '4 ...",Jake_Lamb,https://www.mtgtop8.com/event?e=23922&d=366548...,Rakdos_Cat_Oven
5,"['1 Mountain', '2 Plains', '2 Swamp', '3 Blood...",Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_


In [18]:
decks_clean

,deck_list,player,url,name
0,"['2 Forest', '1 Island', '1 Mountain', '2 Swam...",Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control
1,"['4 Shock', '1 Forest', '1 Island', '1 Mountai...",Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals
2,"['3 Forest', '1 Island', '2 Plains', '1 Swamp'...",Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value
3,"['1 Island', '1 Mountain', '2 Plains', '3 Swam...",Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_
4,"['1 Island', '1 Mountain', '2 Plains', '3 Swam...",Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23442&d=361922...,Mardu_DOOM
...,...,...,...,...
488,"['4 Planar Cleansing', '7 Island', '5 Plains',...",Yokokawa_Yuuta,https://www.mtgtop8.com/event?e=23907&d=366416...,UW_Control
489,"['1 Disenchant', '7 Island', '4 Plains', '4 Ha...",zer1997,https://www.mtgtop8.com/event?e=23772&d=365157...,UW_Control
490,"['17 Plains', '4 Loyal Pegasus', '4 Faerie Gui...",Brandon_Krussow,https://www.mtgtop8.com/event?e=23943&d=366839...,White_Weenie
491,"['2 Disenchant', '17 Plains', '4 Loyal Pegasus...",Nematov_Ruslan,https://www.mtgtop8.com/event?e=23578&d=363291...,White_Weenie


In [19]:
decks

,deck_list,player,url,name
0,"['2 Forest', '1 Island', '1 Mountain', '2 Swam...",Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control
1,"['4 Shock', '1 Forest', '1 Island', '1 Mountai...",Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals
2,"['3 Forest', '1 Island', '2 Plains', '1 Swamp'...",Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value
3,"['1 Island', '1 Mountain', '2 Plains', '3 Swam...",Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_
4,"['1 Island', '1 Mountain', '2 Plains', '3 Swam...",Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23442&d=361922...,Mardu_DOOM
...,...,...,...,...
488,"['4 Planar Cleansing', '7 Island', '5 Plains',...",Yokokawa_Yuuta,https://www.mtgtop8.com/event?e=23907&d=366416...,UW_Control
489,"['1 Disenchant', '7 Island', '4 Plains', '4 Ha...",zer1997,https://www.mtgtop8.com/event?e=23772&d=365157...,UW_Control
490,"['17 Plains', '4 Loyal Pegasus', '4 Faerie Gui...",Brandon_Krussow,https://www.mtgtop8.com/event?e=23943&d=366839...,White_Weenie
491,"['2 Disenchant', '17 Plains', '4 Loyal Pegasus...",Nematov_Ruslan,https://www.mtgtop8.com/event?e=23578&d=363291...,White_Weenie


## Assessment
- [done] deck_list column has , for delimiter but names can also have a comma
- [done] cards are in one column
- [done] multiple cards are in one object
- [done] card number and card name are the same object
- [done] null values in `deck_list` column
- [done] rarities not present in the data
- [done] some card names still have a comma at the end
- some decks have multiple observations
- [done] many cards have nan values for rarity
- [done] dual cards are considered two cards in standard_sets_clean but only one card in decks_clean

# Clean

## Define
- filter for any lists in `deck_list` that have null values in it

## Code

In [20]:
decks_clean.deck_list = decks_clean[decks_clean.astype(str)['deck_list'] != "['nan']"]

## Test

In [21]:
decks_clean[decks_clean.astype(str)['deck_list'] == "['nan']"]

,deck_list,player,url,name


## Define
- make , in `deck_list` into '|' without mistaking commas in names

## Code

In [22]:
decks_clean.deck_list = decks_clean.deck_list.astype('str')
decks_clean.deck_list = decks_clean.deck_list.str.replace(', (?=\d)', '|')

## Define
- split `deck_list` by card

## Code

In [23]:
new = decks_clean.deck_list.str.split('|', expand=True)
decks_clean = decks_clean.join(new)

## Test

In [24]:
decks_clean.head()

,deck_list,player,url,name,0
0,"['2 Forest', '1 Island', '1 Mountain', '2 Swam...",Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,"['2 Forest', '1 Island', '1 Mountain', '2 Swam..."
1,"['4 Shock', '1 Forest', '1 Island', '1 Mountai...",Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,"['4 Shock', '1 Forest', '1 Island', '1 Mountai..."
2,"['3 Forest', '1 Island', '2 Plains', '1 Swamp'...",Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,"['3 Forest', '1 Island', '2 Plains', '1 Swamp'..."
3,"['1 Island', '1 Mountain', '2 Plains', '3 Swam...",Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,"['1 Island', '1 Mountain', '2 Plains', '3 Swam..."
4,"['1 Island', '1 Mountain', '2 Plains', '3 Swam...",Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23442&d=361922...,Mardu_DOOM,"['1 Island', '1 Mountain', '2 Plains', '3 Swam..."


## Define 
- melt cards into one column

## Code

In [25]:
decks_clean = pd.melt(decks_clean, id_vars=['deck_list', 'player', 'url', 'name'], value_vars=decks_clean.columns[4:45])

In [26]:
decks_clean = decks_clean.rename(columns={'value':'cards'}).drop(columns=['deck_list', 'variable'])

## Test

In [27]:
decks_clean.shape

(493, 4)

In [28]:
decks_clean.head()

,player,url,name,cards
0,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,"['2 Forest', '1 Island', '1 Mountain', '2 Swam..."
1,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,"['4 Shock', '1 Forest', '1 Island', '1 Mountai..."
2,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,"['3 Forest', '1 Island', '2 Plains', '1 Swamp'..."
3,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,"['1 Island', '1 Mountain', '2 Plains', '3 Swam..."
4,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23442&d=361922...,Mardu_DOOM,"['1 Island', '1 Mountain', '2 Plains', '3 Swam..."


## Define
- drop rows that have 'None' in them

## Code

In [29]:
decks_clean.cards = decks_clean.cards.astype('str')

In [30]:
decks_clean = decks_clean[decks_clean.cards != 'None']

In [32]:
decks_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/decks_clean.csv', index=False)

## Test

In [35]:
decks_clean[decks_clean.cards == 'None']

,player,url,name,cards


## Define
- get numbers out of `cards` column

## Code

In [36]:
new = decks_clean.cards.str.split('(?<=\d)\s', expand=True)
decks_clean = decks_clean.join(new).rename(columns={'name':'deck_name', 0:'num', 1:'card_name'}).drop(columns=['cards'])

## Test

In [37]:
decks_clean.head()

,player,url,deck_name,num,card_name,2,3,4,5,6,...,31,32,33,34,35,36,37,38,39,40
0,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,['2,"Forest', '1","Island', '1","Mountain', '2","Swamp', '3","Breeding Pool', '2","Mystical Dispute', '2",...,None,None,None,None,None,None,None,None,None,None
1,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,['4,"Shock', '1","Forest', '1","Island', '1","Mountain', '1","Swamp', '1","Blood Crypt', '3",...,"Overgrown Tomb', '2","Watery Grave', '1","Temple of Mystery', '3","Neoform, ']",None,None,None,None,None,None
2,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,['3,"Forest', '1","Island', '2","Plains', '1","Swamp', '4","Breeding Pool', '4","Gilded Goose', '2",...,None,None,None,None,None,None,None,None,None,None
3,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,['1,"Island', '1","Mountain', '2","Plains', '3","Swamp', '2","Blood Crypt', '1","Hallowed Fountain', '3",...,"Oath of Kaya', '4","Guild Globe', '2","Sorcerous Spyglass, ']",None,None,None,None,None,None,None
4,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23442&d=361922...,Mardu_DOOM,['1,"Island', '1","Mountain', '2","Plains', '3","Swamp', '2","Blood Crypt', '1","Hallowed Fountain', '3",...,"Oath of Kaya', '4","Guild Globe', '2","Sorcerous Spyglass, ']",None,None,None,None,None,None,None


In [39]:
decks_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/decks_clean.csv', index=False)

## Define
- Add card rarity to decks_clean

In [42]:
# code to get rarity in a list by themselves
card_rarities = []
for i in standard_sets_clean.keys():
    current_set = standard_sets_clean[i]['cards']
    for a in np.arange(np.count_nonzero(current_set)):
        rarity = current_set[a]['rarity']
        card_rarities.append(rarity)

In [43]:
# code to get names in a list by themselves
card_names = []
for i in standard_sets_clean.keys():
    current_set = standard_sets_clean[i]['cards']
    for a in np.arange(np.count_nonzero(current_set)):
        name = current_set[a]['name']
        card_names.append(name)

In [44]:
# code to create dataframe of card_rarities and card_names
mtg_card_rarities = pd.DataFrame({'card_name': card_names, 'card_rarity': card_rarities})
mtg_card_rarities_clean = mtg_card_rarities.copy()
mtg_card_rarities_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/mtg_card_rarities_clean.csv', index=False)

In [45]:
# drop duplicates in mtg_card_rarities_clean
mtg_card_rarities_clean.drop_duplicates(inplace=True)

In [46]:
mtg_card_rarities_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/mtg_card_rarities_clean.csv', index=False)

In [47]:
mtg_card_rarities_clean = pd.read_csv('~/Documents/MTGA_rarity_analysis/data/mtg_card_rarities_clean.csv')

In [48]:
decks_clean = decks_clean.join(mtg_card_rarities_clean.set_index('card_name'), on='card_name')

## Test

In [49]:
mtg_card_rarities_clean

,card_name,card_rarity
0,Acclaimed Contender,rare
1,"Alela, Artful Provocateur",mythic
2,All That Glitters,uncommon
3,Bring to Life,uncommon
4,Animating Faerie,uncommon
...,...,...
1415,Wall of Runes,common
1416,Wanderer's Strike,common
1417,War Screecher,common
1418,Wardscale Crocodile,common


In [50]:
decks_clean

,player,url,deck_name,num,card_name,2,3,4,5,6,...,32,33,34,35,36,37,38,39,40,card_rarity
0,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,['2,"Forest', '1","Island', '1","Mountain', '2","Swamp', '3","Breeding Pool', '2","Mystical Dispute', '2",...,None,None,None,None,None,None,None,None,None,NaN
1,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,['4,"Shock', '1","Forest', '1","Island', '1","Mountain', '1","Swamp', '1","Blood Crypt', '3",...,"Watery Grave', '1","Temple of Mystery', '3","Neoform, ']",None,None,None,None,None,None,NaN
2,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,['3,"Forest', '1","Island', '2","Plains', '1","Swamp', '4","Breeding Pool', '4","Gilded Goose', '2",...,None,None,None,None,None,None,None,None,None,NaN
3,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,['1,"Island', '1","Mountain', '2","Plains', '3","Swamp', '2","Blood Crypt', '1","Hallowed Fountain', '3",...,"Guild Globe', '2","Sorcerous Spyglass, ']",None,None,None,None,None,None,None,NaN
4,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23442&d=361922...,Mardu_DOOM,['1,"Island', '1","Mountain', '2","Plains', '3","Swamp', '2","Blood Crypt', '1","Hallowed Fountain', '3",...,"Guild Globe', '2","Sorcerous Spyglass, ']",None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,Yokokawa_Yuuta,https://www.mtgtop8.com/event?e=23907&d=366416...,UW_Control,['4,"Planar Cleansing', '7","Island', '5","Plains', '4","Hallowed Fountain', '3","Giant Killer', '4","Brazen Borrower', '2",...,None,None,None,None,None,None,None,None,None,NaN
489,zer1997,https://www.mtgtop8.com/event?e=23772&d=365157...,UW_Control,['1,"Disenchant', '7","Island', '4","Plains', '4","Hallowed Fountain', '3","Glass Casket', '1","Realm-Cloaked Giant', '4",...,None,None,None,None,None,None,None,None,None,NaN
490,Brandon_Krussow,https://www.mtgtop8.com/event?e=23943&d=366839...,White_Weenie,['17,"Plains', '4","Loyal Pegasus', '4","Faerie Guidemother', '4","Giant Killer', '1","Shepherd of the Flock', '1","Castle Ardenvale', '4",...,None,None,None,None,None,None,None,None,None,NaN
491,Nematov_Ruslan,https://www.mtgtop8.com/event?e=23578&d=363291...,White_Weenie,['2,"Disenchant', '17","Plains', '4","Loyal Pegasus', '4","Faerie Guidemother', '2","Giant Killer', '4","Glass Casket', '3",...,None,None,None,None,None,None,None,None,None,NaN


In [51]:
decks_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 493 entries, 0 to 492
Data columns (total 45 columns):
player         493 non-null object
url            493 non-null object
deck_name      492 non-null object
num            493 non-null object
card_name      492 non-null object
2              492 non-null object
3              492 non-null object
4              492 non-null object
5              492 non-null object
6              492 non-null object
7              492 non-null object
8              492 non-null object
9              492 non-null object
10             492 non-null object
11             492 non-null object
12             492 non-null object
13             492 non-null object
14             492 non-null object
15             491 non-null object
16             491 non-null object
17             489 non-null object
18             481 non-null object
19             471 non-null object
20             457 non-null object
21             439 non-null object
22             409 n

In [52]:
type(decks_clean.card_rarity[0])

float

In [53]:
decks_clean.sample(10)

,player,url,deck_name,num,card_name,2,3,4,5,6,...,32,33,34,35,36,37,38,39,40,card_rarity
328,Noireve_Sliver,https://www.mtgtop8.com/event?e=23925&d=366569...,Rakdos_Aggro,['1,"Act of Treason', '7","Mountain', '6","Swamp', '4","Blood Crypt', '4","Cauldron Familiar', '1","Murderous Rider', '3",...,None,None,None,None,None,None,None,None,None,NaN
402,Simone_Rocutto,https://www.mtgtop8.com/event?e=23942&d=366828...,Simic_Ramp,['7,"Forest', '4","Island', '4","Breeding Pool', '4","Mystical Dispute', '4","Gilded Goose', '2","Castle Garenbrig', '2",...,None,None,None,None,None,None,None,None,None,NaN
172,Qail_Shivji,https://www.mtgtop8.com/event?e=23478&d=362239...,Golos,['2,"Forest', '2","Island', '2","Swamp', '2","Breeding Pool', '1","Once Upon a Time', '3","Oko, Thief of Crowns', '3",...,"Duress', '2","Arboreal Grazer', '1","Ashiok, Dream Render', '3","Disfigure, ']",None,None,None,None,None,NaN
45,Jessica_Estephan,https://www.mtgtop8.com/event?e=23371&d=361242...,Bant_Aggro_PW,['2,"Shock', '10","Island', '4","Mountain', '4","Brazen Borrower', '3","Gadwick, the Wizened', '2","Mystical Dispute', '4",...,None,None,None,None,None,None,None,None,None,NaN
215,Calla_Adrien,https://www.mtgtop8.com/event?e=23924&d=366561...,Izzet_Flash,['2,"Shock', '8","Island', '5","Mountain', '4","Brazen Borrower', '3","Gadwick, the Wizened', '3","Mystical Dispute', '4",...,None,None,None,None,None,None,None,None,None,NaN
249,Artemiy_Kosarev,https://www.mtgtop8.com/event?e=23945&d=366861...,Jund_Sacrifice,['4,"Forest', '1","Mountain', '3","Swamp', '4","Blood Crypt', '4","Cauldron Familiar', '2","Epic Downfall', '2",...,None,None,None,None,None,None,None,None,None,NaN
33,Andrea_Mengucci,https://www.mtgtop8.com/event?e=23371&d=361213...,Bant_Aggro,['10,"Forest', '6","Island', '4","Breeding Pool', '3","Mystical Dispute', '2","Castle Garenbrig', '2","Castle Vantress', '2",...,None,None,None,None,None,None,None,None,None,NaN
152,Théo_Moutier,https://www.mtgtop8.com/event?e=23942&d=366832...,Golgari_Sacrifice,['9,"Forest', '7","Swamp', '3","Cauldron Familiar', '4","Murderous Rider', '4","Gilded Goose', '4","Trail of Crumbs', '4",...,None,None,None,None,None,None,None,None,None,NaN
153,Theshaman,https://www.mtgtop8.com/event?e=23874&d=366102...,Golgari,['6,"Forest', '7","Swamp', '3","Cauldron Familiar', '4","Foulmire Knight', '4","Murderous Rider', '1","Order of Midnight', '4",...,None,None,None,None,None,None,None,None,None,NaN
150,Stanislav_Cifka,https://www.mtgtop8.com/event?e=23942&d=366829...,Golgari_Sacrifice,['7,"Forest', '8","Swamp', '4","Cauldron Familiar', '1","Epic Downfall', '4","Murderous Rider', '4","Gilded Goose', '4",...,None,None,None,None,None,None,None,None,None,NaN


In [55]:
decks_clean.card_name.nunique()

60

In [56]:
mtg_card_rarities_clean.shape

(1420, 2)

In [57]:
dc_rarity_null = decks_clean[decks_clean.card_rarity.isna()]

In [58]:
dc_rarity_null.card_name.value_counts()

Forest', '1                80
Forest', '2                62
Forest', '6                30
Forest', '7                28
Forest', '8                22
Shock', '10                20
Forest', '5                18
Forest', '3                17
Island', '4                15
Disenchant', '2            14
Shock', '9                 11
Swamp', '3                  9
Shock', '18                 9
Planar Cleansing', '8       9
Planar Cleansing', '1       8
Island', '2                 8
Shock', '1                  7
Mountain', '1               7
Planar Cleansing', '2       7
Island', '1                 7
Forest', '4                 6
Mountain', '9               6
Mountain', '8               5
Shock', '8                  5
Shock', '6                  5
Planar Cleansing', '3       5
Swamp', '2                  5
Mountain', '3               4
Planar Cleansing', '6       4
Forest', '9                 4
Mountain', '7               4
Tectonic Rift', '2          4
Disenchant', '7             4
Planar Cle

In [59]:
decks_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/decks_clean.csv', index=False)

## Define
- get rid of nan in `card_name`

## Code 

In [60]:
decks_clean.card_name = decks_clean.card_name.str.replace(', $', '')

In [61]:
decks_clean = decks_clean.join(mtg_card_rarities_clean.set_index('card_name'), on='card_name', lsuffix='x').drop(columns='card_rarityx')

In [62]:
dual_cards = decks_clean[decks_clean.card_name.str.contains('.+-.+') & decks_clean.card_rarity.isna()]
new = dual_cards.card_name.str.split('-', expand=True)
dual_cards = dual_cards.join(new)

In [63]:
dual_cards = dual_cards.melt(id_vars=list(decks_clean.columns), value_vars=[0, 1], value_name='card_name_2').drop(columns=['variable'])

KeyError: "The following 'id_vars' are not present in the DataFrame: ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '5', '6', '7', '8', '9']"

In [ ]:
decks_clean = decks_clean.join(dual_cards[['card_name', 'card_name_2']].set_index('card_name'), on='card_name')

In [ ]:
decks_clean.card_rarity = decks_clean.card_rarity.astype('str')
for i in np.arange(decks_clean.shape[0]):
    if decks_clean.card_name[i] == 'Expansion-Explosion':
        re.sub('nan', 'rare', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Collision-Colossus':
        re.sub('nan', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Flower-Flourish':
        re.sub('nan', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Thrash-Threat':
        re.sub('nan', 'rare', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Discovery-Dispersal':
        re.sub('nan', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Stasus-Statue':
        re.sub('nan', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Bedeck-Bedazzle':
        re.sub('nan', 'rare', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Consecrate-Consume':
        re.sub('nan', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Carnival-Carnage':
        re.sub('nan', 'uncommon', decks_clean.card_rarity[i])

In [ ]:
decks_clean.card_rarity = decks_clean.card_rarity.astype('str')
for i in np.arange(decks_clean.shape[0]):
    if decks_clean.card_name[i] == 'Expansion-Explosion':
        decks_clean.card_rarity[i] = re.sub('nan', 'rare', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Collision-Colossus':
        decks_clean.card_rarity[i] = re.sub('nan', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Flower-Flourish':
        decks_clean.card_rarity[i] = re.sub('nan', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Thrash-Threat':
        decks_clean.card_rarity[i] = re.sub('nan', 'rare', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Discovery-Dispersal':
        decks_clean.card_rarity[i] = re.sub('nan', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Stasus-Statue':
        decks_clean.card_rarity[i] = re.sub('nan', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Bedeck-Bedazzle':
        decks_clean.card_rarity[i] = re.sub('nan', 'rare', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Consecrate-Consume':
        decks_clean.card_rarity[i] = re.sub('nan', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Carnival-Carnage':
        decks_clean.card_rarity[i] = re.sub('nan', 'uncommon', decks_clean.card_rarity[i])

In [ ]:
conditions = [(decks_clean.card_name == 'Expansion-Explosion'), 
              (decks_clean.card_name == 'Collision-Colossus'), 
              (decks_clean.card_name == 'Flower-Flourish'),
              (decks_clean.card_name == 'Thrash-Threat'),
              (decks_clean.card_name == 'Discovery-Dispersal'),
              (decks_clean.card_name == 'Stasus-Statue'),
              (decks_clean.card_name == 'Bedeck-Bedazzle'),
              (decks_clean.card_name == 'Consecrate-Consume'),
              (decks_clean.card_name == 'Carnival-Carnage')]

choices = ('rare', 'uncommon', 'uncommon', 'rare', 'uncommon', 'uncommon', 'rare', 'uncommon', 'uncommon')

decks_clean['split_card_rarity'] = np.select(conditions, choices)

In [ ]:
decks_clean.card_rarity.dropna(inplace=True)

## Test

In [64]:
decks_clean

,player,url,deck_name,num,card_name,2,3,4,5,6,...,32,33,34,35,36,37,38,39,40,card_rarity
0,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,['2,"Forest', '1","Island', '1","Mountain', '2","Swamp', '3","Breeding Pool', '2","Mystical Dispute', '2",...,None,None,None,None,None,None,None,None,None,NaN
1,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,['4,"Shock', '1","Forest', '1","Island', '1","Mountain', '1","Swamp', '1","Blood Crypt', '3",...,"Watery Grave', '1","Temple of Mystery', '3","Neoform, ']",None,None,None,None,None,None,NaN
2,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,['3,"Forest', '1","Island', '2","Plains', '1","Swamp', '4","Breeding Pool', '4","Gilded Goose', '2",...,None,None,None,None,None,None,None,None,None,NaN
3,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,['1,"Island', '1","Mountain', '2","Plains', '3","Swamp', '2","Blood Crypt', '1","Hallowed Fountain', '3",...,"Guild Globe', '2","Sorcerous Spyglass, ']",None,None,None,None,None,None,None,NaN
4,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23442&d=361922...,Mardu_DOOM,['1,"Island', '1","Mountain', '2","Plains', '3","Swamp', '2","Blood Crypt', '1","Hallowed Fountain', '3",...,"Guild Globe', '2","Sorcerous Spyglass, ']",None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,Yokokawa_Yuuta,https://www.mtgtop8.com/event?e=23907&d=366416...,UW_Control,['4,"Planar Cleansing', '7","Island', '5","Plains', '4","Hallowed Fountain', '3","Giant Killer', '4","Brazen Borrower', '2",...,None,None,None,None,None,None,None,None,None,NaN
489,zer1997,https://www.mtgtop8.com/event?e=23772&d=365157...,UW_Control,['1,"Disenchant', '7","Island', '4","Plains', '4","Hallowed Fountain', '3","Glass Casket', '1","Realm-Cloaked Giant', '4",...,None,None,None,None,None,None,None,None,None,NaN
490,Brandon_Krussow,https://www.mtgtop8.com/event?e=23943&d=366839...,White_Weenie,['17,"Plains', '4","Loyal Pegasus', '4","Faerie Guidemother', '4","Giant Killer', '1","Shepherd of the Flock', '1","Castle Ardenvale', '4",...,None,None,None,None,None,None,None,None,None,NaN
491,Nematov_Ruslan,https://www.mtgtop8.com/event?e=23578&d=363291...,White_Weenie,['2,"Disenchant', '17","Plains', '4","Loyal Pegasus', '4","Faerie Guidemother', '2","Giant Killer', '4","Glass Casket', '3",...,None,None,None,None,None,None,None,None,None,NaN


In [ ]:
decks_clean.card_rarity.isna().sum()

In [ ]:
dc_rarity_null = decks_clean[decks_clean.card_rarity.isna()]

In [ ]:
dc_rarity_null.shape[0]

In [ ]:
dc_rarity_null.card_name.value_counts()

In [ ]:
standard_sets_clean.keys()

In [ ]:
names = []
for i in standard_sets_clean['GRN']['cards']:
    names.append(i['name'])

In [ ]:
names

In [ ]:
decks_clean[decks_clean.card_name == 'Massacre']

In [ ]:
dual_cards.head()

In [ ]:
dual_cards[0]

In [ ]:
list(decks_clean.columns)

In [ ]:
decks_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/decks_clean.csv', index=False)

In [ ]:
decks_clean[decks_clean.card_rarity == 'nan']

In [ ]:
decks_clean.split_card_rarity.unique()

In [ ]:
decks_clean.query('split_card_rarity == "rare" or split_card_rarity == "uncommon"')

In [ ]:
decks_clean = decks_clean.melt(id_vars=['player', 'url', 'deck_name', 'num', 'card_name'], value_vars=['card_rarity', 'split_card_rarity'], value_name='card_rarity')

In [ ]:
decks_clean.card_rarity.dropna(inplace=True)

In [ ]:
decks_clean.duplicated().sum()

In [ ]:
decks_clean.drop(columns=['variable'], inplace=True)

In [ ]:
decks_clean.drop_duplicates(inplace=True)

In [ ]:
decks_clean = decks_clean[decks_clean.card_rarity != '0']

In [ ]:
decks_clean.iloc[10844].card_rarity

In [ ]:
decks_clean.shape

In [ ]:
decks_clean = decks_clean[decks_clean.card_rarity != 'nan']

In [ ]:
decks_clean.shape

In [ ]:
decks_clean.card_rarity.unique()

In [ ]:
decks_clean[decks_clean.player.duplicated()]

In [ ]:
decks_clean.groupby(['player', 'deck_name']).num.sum()[decks_clean.groupby(['player', 'deck_name']).num.sum().ge(76)]

In [ ]:
decks_clean.query("player in 'VTCLA' and deck_name in 'Adventure'")